<a href="https://colab.research.google.com/github/Expert-Han/Nonlinear_Model_Predictive_Control_Python_byHan/blob/main/NMPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Python version check in Colab & Casadi install

In [2]:
!python --version

Python 3.8.10


In [3]:
pip install casadi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 53.0 MB/s eta 0:00:00


In [4]:
from casadi import *
x = MX.sym("x")
print(jacobian(sin(x),x))

cos(x)


## Main code for NMPC (Nonlinear Model Predictive Control)

In [ ]:
import numpy as np
# Q = np.array([1, 0, 0],[0, 1, 0],[0, 0, 1])
Q = np.array([1, 0, 0],[0, 1, 0],[0, 0, 1]) # weighting matrix for state
R = np.array([0.1]) # weighting matrix for input

# x_cl_out{1} = x_cl{1};
# u_cl_out{1} = u_cl{1};

M = 0.5;      % mass of the cart (kg)  
m = 0.5;      % mass of the pendulum bob (kg)
L = 1;        % length of the pendulum rod (m)
g = 9.81;
im = 0.5;      % inertia  

a21 = (m*g*(L/2))/(im+m*(L/2)^2-((m^2*(L/2)^2)/(m+M)));
a41 = ((im+m*(L/2)^2)*g)*(im+m*(L/2)^2-(m^2*(L/2)^2)/(M+m));
b2 = ((m*(L/2))/(M+m))/(im+m*(L/2)^2-(m^2*(L/2)^2)/(M+m));
b4 = ((im+m*(L/2)^2)/(M+m))/(im+m*(L/2)^2-(m^2*(L/2)^2)/(M+m));

A = [0 1 0 0;
    a21 0 0 0;
    0 0 0 1;
    a41 0 0 0];
B = [0;
    b2;
    0;
    b4];
    
x1 = MX.sym('x1'); # States define
x2 = MX.sym('x2');
x  = [x1; x2];    
u  = MX.sym('u'); # Controls

# Van der Pol oscillator system  (nonlinear ODE)
xbar=2;
A1 = [1-xbar^2 -1;1 0];
A2 = [1 -1; 1 0];
B = [1; 0];
% Van der Pol oscillator system  (nonlinear ODE)
th1 = (x2^2)/xbar^2;
th2 = 1-th1;

ode = th1*(A1*x+B*u)+th2*(A2*x+B*u);

f = Function('f',{x,u},{ode},{'x','u'},{'ode'});
intg_options = struct;
intg_options.tf = ti;
intg_options.simplify = true;

dae = struct;
dae.x = x;         % What are states?
dae.p = u;         % What are parameters (=fixed during the integration horizon)?
dae.ode = f(x,u);  % Expression for the right-hand side

intg = integrator('intzg','rk',dae,intg_options);
res = intg('x0',x,'p',u); % Evaluate with symbols
x_next = res.xf;
F = Function('F',{x,u},{x_next},{'x','u'},{'x_next'});    

hmax = h;
hmin = 0.01;
var_sam = [hmin:ti:hmax];   
% save_h = [0.7 0.2 0.4 0.5 0.4 0.6 0.3 0.7 0.3];      
ran_num = randi([1 size(var_sam,2)],1);
hj = var_sam(ran_num);

while (j <= Iterations)

    t = 1;      % Initialize time
    data_h(1)=0;
    Dn = 1;
    x_gMPC = x0;  % Initial condition
    num_a = 0;

    [~, uPred] = solve_CFTOCP(x_gMPC(:,t), N, Q, R, X, U, ti, hmax);
    sam_u = uPred;
    u_gMPC(:,1) = uPred;
  
    tollerance = 10^(-9);
    exitFlag = 0;
    sam_s = x_gMPC(:,1);
    s_Sample(:,1) = sam_s;
    
    while ( exitFlag == 0 )
        clc
        fprintf('Time step: %d, Iteration: %d, Cost: %13.15f\n', [t, j, IterationCost_out{j}(1)]);
        uPred
        update_t = round(hj/ti);
%         disp(['Time step: ', num2str(t), ', Iteration: ', num2str(j), ' Best Cost: ',num2str(IterationCost_out{j}(1))])
        if num_a == (update_t)
                ran_num = randi([1 size(var_sam,2)],1);
                hj = var_sam(ran_num);
                [~, uPred] = solve_CFTOCP(x_gMPC(:,t), N, Q, R, X, U, ti, hmax);
                sam_s = x_gMPC(:,t);
                num_a = 0;
                Dn = Dn + 1;
                data_h(Dn) = t*ti;      
        end

        % Update system position
        u_gMPC(:,t+1) = uPred;
        s_Sample(:,t+1) = sam_s;
        temp_cost_v(t) = cost_v;
        x_gMPC(:,t+1) = full(F(x_gMPC(:,t),uPred));
        num_a = num_a + 1;
        t = t + 1;
        % Check exits conditions
        if x_gMPC(:,t)'*x_gMPC(:,t) < (tollerance)
            exitFlag = 1;
        end
    end
    
    % Now save the data, update cost and safe set.
    x_cl_out{j+1} = s_Sample;
    u_cl_out{j+1} = u_gMPC;
    data_h_out{j} = data_h;
    IterationCost_out{j+1} = ComputeCost(s_Sample, u_LMPC, Q, R);   
    % increase Iteration index and restart
    j = j + 1;
    if j <= 1
        clear x_gMPC
        clear u_gMPC
        clear sam_s
        clear s_Sample
        clear data_h
    end
    
end

end